# Libs

In [42]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import psycopg2 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import os

load_dotenv()

In [43]:
USER = os.environ.get('POSTGRES_USER')
PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB = os.environ.get('POSTGRES_DB_netology')

conn_string = psycopg2.connect(f"dbname={DB} user='{USER}' host='localhost' password='{PASSWORD}'")
cursor = conn_string.cursor()

# Exercise 1

In [44]:
# --1. Какие самолеты имеют более 50 посадочных мест?
# # --1. Which airplanes have more than 50 seats?

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = ('    select model AS "самолет", s.aircraft_code as "код самолета",' +  
 	                'count(s.seat_no) as "кол-во мест"' +
            ' from seats s ' +
            ' join aircrafts ad on ad.aircraft_code = s.aircraft_code' +
            ' group by ad.model, s.aircraft_code ' +
            ' having count(s.seat_no) > 50 ' +
            ' order by count(s.seat_no);')
data = pd.read_sql(query, con=conn_string)
data

,самолет,код самолета,кол-во мест
0,Sukhoi SuperJet-100,SU9,97
1,Airbus A319-100,319,116
2,Boeing 737-300,733,130
3,Airbus A320-200,320,140
4,Airbus A321-200,321,170
5,Boeing 767-300,763,222
6,Boeing 777-300,773,402


In [24]:
aircrafts = (f'select * from aircrafts')
aircrafts = pd.read_sql(aircrafts, con=conn_string)
seats = (f'select * from seats')
seats = pd.read_sql(seats, con=conn_string)

result = aircrafts.set_index('aircraft_code').join(seats.set_index('aircraft_code')).groupby('model').size().reset_index(name='count')
result[result['count'] > 50].sort_values('count', ascending=False).reset_index(drop=True).rename(columns={'model': 'самолет', 'count': 'кол-во мест'})

,самолет,кол-во мест
0,Boeing 777-300,402
1,Boeing 767-300,222
2,Airbus A321-200,170
3,Airbus A320-200,140
4,Boeing 737-300,130
5,Airbus A319-100,116
6,Sukhoi SuperJet-100,97


# Exercise 2

In [18]:
# --2. В каких аэропортах есть рейсы, в рамках которых можно добраться бизнес-классом дешевле, чем эконом-классом?
# --2. In which airports are there flights where you can get to business class cheaper than economy class?

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = ('   with ec as ' + 
            '   (select amount, f.flight_id, f.departure_airport,fare_conditions ' +
            '   from flights f ' +
            '   join ticket_flights tf on tf.flight_id = f.flight_id ' +
            "   where fare_conditions = 'Economy' order by 3), " +
            '   bc as' + 
            '    (select amount, f.flight_id, f.departure_airport,fare_conditions ' +
            '   from flights f ' +
            '   join ticket_flights tf on tf.flight_id = f.flight_id ' +  
            "   where fare_conditions = 'Business' order by 3) " +
            'select distinct ec.departure_airport, ' + 
            'bc.flight_id as "bus_flight_id", bc.amount as "bus_price", ' +
            'ec.flight_id as "eco_flight_id", ec.amount as "eco_flight" ' +
            'from ec ' +
            'join bc on bc.flight_id = ec.flight_id ' +
            'where bc.amount < ec.amount;')

data = pd.read_sql(query, con=conn_string)
data

,departure_airport,bus_flight_id,bus_price,eco_flight_id,eco_flight


In [35]:
flights = (f'select * from flights')
flights = pd.read_sql(flights, con=conn_string)

ticket_flights = (f'select * from ticket_flights')
ticket_flights = pd.read_sql(ticket_flights, con=conn_string)
ticket_flights

,ticket_no,flight_id,fare_conditions,amount
0,0005432159776,30625,Business,42100.0
1,0005435212351,30625,Business,42100.0
2,0005435212386,30625,Business,42100.0
3,0005435212381,30625,Business,42100.0
4,0005432211370,30625,Business,42100.0
...,...,...,...,...
1045721,0005435097522,32094,Economy,5200.0
1045722,0005435097521,32094,Economy,5200.0
1045723,0005435104384,32094,Economy,5200.0
1045724,0005435104352,32094,Economy,5200.0


In [36]:
flights

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1,PG0405,2016-09-13 05:35:00+00:00,2016-09-13 06:30:00+00:00,DME,LED,Arrived,321,2016-09-13 05:44:00+00:00,2016-09-13 06:39:00+00:00
1,2,PG0404,2016-10-03 15:05:00+00:00,2016-10-03 16:00:00+00:00,DME,LED,Arrived,321,2016-10-03 15:06:00+00:00,2016-10-03 16:01:00+00:00
2,3,PG0405,2016-10-03 05:35:00+00:00,2016-10-03 06:30:00+00:00,DME,LED,Arrived,321,2016-10-03 05:39:00+00:00,2016-10-03 06:34:00+00:00
3,4,PG0402,2016-11-07 08:25:00+00:00,2016-11-07 09:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
4,5,PG0405,2016-10-14 05:35:00+00:00,2016-10-14 06:30:00+00:00,DME,LED,On Time,321,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
33116,33117,PG0063,2016-09-30 15:25:00+00:00,2016-09-30 16:10:00+00:00,SKX,SVO,Arrived,CR2,2016-09-30 15:25:00+00:00,2016-09-30 16:10:00+00:00
33117,33118,PG0063,2016-09-25 15:25:00+00:00,2016-09-25 16:10:00+00:00,SKX,SVO,Arrived,CR2,2016-09-25 15:30:00+00:00,2016-09-25 16:15:00+00:00
33118,33119,PG0063,2016-11-06 15:25:00+00:00,2016-11-06 16:10:00+00:00,SKX,SVO,Scheduled,CR2,NaT,NaT
33119,33120,PG0063,2016-09-29 15:25:00+00:00,2016-09-29 16:10:00+00:00,SKX,SVO,Arrived,CR2,2016-09-29 15:26:00+00:00,2016-09-29 16:12:00+00:00


# Exercise 3

In [17]:
# --3.Есть ли самолеты, не имеющие бизнес-класса?
# --3. Are there airplanes without business class?

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = ('   select ad.model, ad.aircraft_code, t.fare_class ' +
            'from ( select aircraft_code, array_agg(fare_conditions) as "fare_class" ' +
            ' 	from seats group by 1) t ' +
            'join aircrafts ad on t.aircraft_code = ad.aircraft_code ' + 
            "where not 'Business' = any (t.fare_class);")

data = pd.read_sql(query, con=conn_string)
data

,model,aircraft_code,fare_class
0,Cessna 208 Caravan,CN1,"[Economy, Economy, Economy, Economy, Economy, ..."
1,Bombardier CRJ-200,CR2,"[Economy, Economy, Economy, Economy, Economy, ..."


In [38]:
aircrafts = (f'select * from aircrafts')
aircrafts = pd.read_sql(aircrafts, con=conn_string)
seats = (f'select * from seats')
seats = pd.read_sql(seats, con=conn_string)
seats

,aircraft_code,seat_no,fare_conditions
0,319,2A,Business
1,319,2C,Business
2,319,2D,Business
3,319,2F,Business
4,319,3A,Business
...,...,...,...
1334,773,48H,Economy
1335,773,48K,Economy
1336,773,49A,Economy
1337,773,49C,Economy


In [39]:
aircrafts

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
6,733,Boeing 737-300,4200
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


# Exercise 4

In [29]:
# --4.Найдите количество занятых мест для каждого рейса, процентное отношение 
# --количества занятых мест к общему количеству мест в самолете, добавьте 
# --накопительный итог вывезенных пассажиров по каждому аэропорту на каждый день
# --4.Find the number of occupied seats for each flight, the percentage of occupied seats 
# --to the total number of seats on the plane, add the cumulative total of passengers carried out 
# --for each airport for each day

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   ' with cc as (select ad.aircraft_code, count(seat_no) as seats_total ' +
                        ' from seats s ' +
                        ' join aircrafts ad on ad.aircraft_code = s.aircraft_code ' +
                        ' group by ad.aircraft_code ) ' +
            ' select f.flight_no, count(bp.seat_no) as "seats_used", cc.seats_total, ' +
                ' round((cast(count(bp.seat_no) as decimal) / cast(cc.seats_total as decimal))*100,0) ' +
                ' as "percent,%", ' +
                ' f.actual_departure, f.departure_airport, ' +
                " sum(count(bp.seat_no)) over (partition by f.departure_airport, date_trunc('day',f.actual_departure::date)) " +
                ' as "cumulative_day", ' +
                " sum(count(bp.seat_no)) over (partition by f.departure_airport, date_trunc('day',f.actual_departure::date) " +
                ' order by f.actual_departure) as "step_day" ' +
            ' from boarding_passes bp ' +
            ' join flights f on f.flight_id = bp.flight_id ' +
            ' join cc on cc.aircraft_code = f.aircraft_code ' +
            ' group by f.flight_no, f.departure_airport, f.actual_departure, cc.seats_total ' +
            ' order by 6, 5 ' +
            ' limit 600')

data = pd.read_sql(query, con=conn_string)
data

,flight_no,seats_used,seats_total,"percent,%",actual_departure,departure_airport,cumulative_day,step_day
0,PG0480,3,97,3.0,2016-09-13 08:35:00+00:00,AAQ,54.0,3.0
1,PG0252,51,130,39.0,2016-09-13 09:08:00+00:00,AAQ,54.0,54.0
2,PG0480,3,97,3.0,2016-09-14 08:29:00+00:00,AAQ,53.0,3.0
3,PG0252,50,130,38.0,2016-09-14 09:07:00+00:00,AAQ,53.0,53.0
4,PG0480,5,97,5.0,2016-09-15 08:26:00+00:00,AAQ,55.0,5.0
...,...,...,...,...,...,...,...,...
595,PG0135,64,116,55.0,2016-10-04 01:30:00+00:00,BTK,64.0,64.0
596,PG0135,59,116,51.0,2016-10-05 01:27:00+00:00,BTK,59.0,59.0
597,PG0135,56,116,48.0,2016-10-06 01:28:00+00:00,BTK,56.0,56.0
598,PG0135,53,116,46.0,2016-10-07 01:28:00+00:00,BTK,53.0,53.0


In [21]:
# --5. Найдите процентное соотношение перелетов по маршрутам от общего количества 
# --перелетов. Выведите в результат названия аэропортов и процентное отношение.
# --5. Find the percentage of flights by route from the total number of flights. 
# --Output the names of airports and the percentage.

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   ' select distinct flight_no, departure_airport, arrival_airport, ' +
            ' round((cast((count(flight_id) over (partition by flight_no)) as decimal) / (count(flight_no) over ()))*100,3) as "percent,%" ' +
            ' from flights f order by 4')

data = pd.read_sql(query, con=conn_string)
data

,flight_no,departure_airport,arrival_airport,"percent,%"
0,PG0001,UIK,SGC,0.024
1,PG0073,KRR,NOZ,0.024
2,PG0088,KHV,DYR,0.024
3,PG0170,YKS,BAX,0.024
4,PG0255,SVO,DYR,0.024
...,...,...,...,...
705,PG0700,MJZ,YKS,0.184
706,PG0703,SVO,UUS,0.184
707,PG0704,UUS,SVO,0.184
708,PG0707,OVS,SGC,0.184


# Exercise 6

In [28]:
# --6.Выведите количество пассажиров по каждому коду сотового оператора, если 
# --учесть, что код оператора - это три символа после +7
# --6. Display the number of passengers for each mobile operator code, 
# --taking into account that the operator code is three characters after +7

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   ' select count(passenger_name) as "pass_quant", ' +
 	        '    (substring((contact_data ->> \'phone\') from 3 for 3)) as "code" ' +
            ' from tickets t ' +
            ' group by (substring((contact_data ->> \'phone\') from 3 for 3)) ' +
            ' order by 1')
data = pd.read_sql(query, con=conn_string)
data

,pass_quant,code
0,3520,046
1,3538,008
2,3540,000
3,3555,090
4,3562,089
...,...,...
95,3800,032
96,3811,091
97,3812,012
98,3815,017


# Exercise 7

In [30]:
# --7.Между какими городами не существует перелетов?
# --7.Between which cities are there no flights?

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   'select distinct ad1.city, ad2.city ' +
            'from airports ad1, airports ad2 ' +
            'where ad1.city != ad2.city ' +
            'except ' +
            'select distinct a.city as "dep_city", a2.city as "arr_city" ' +
            'from flights f ' +
            'join airports a on f.departure_airport = a.airport_code ' +
            'join airports a2 on f.arrival_airport = a2.airport_code  ' +
            'order by 1')
data = pd.read_sql(query, con=conn_string)
data

,city,city
0,Абакан,Геленджик
1,Абакан,Хабаровск
2,Абакан,Сочи
3,Абакан,Когалым
4,Абакан,Курск
...,...,...
9579,Ярославль,Южно-Сахалинск
9580,Ярославль,Комсомольск-на-Амуре
9581,Ярославль,Омск
9582,Ярославль,Усть-Кут


# Exercise 8

In [31]:
# --8.Классифицируйте финансовые обороты (сумма стоимости билетов) по маршрутам:
# --До 50 млн - low
# --От 50 млн включительно до 150 млн - middle
# --От 150 млн включительно - high
# --8. Classify financial turnover (ticket price total) by routes:
# --Up to 50 million - low
# --From 50 million inclusive to 150 million - middle
# --From 150 million inclusive - high

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   ' select classification, count(income_route) ' +
            ' from ( ' +
                ' select flight_no, sum(amount) as "income_route", ' +
            	" case when sum(amount) >= 150000000 then 'high' " +
            		' when sum(amount) < 150000000 and ' +
               			  " sum(amount) >= 50000000 then 'middle' " +
             			  " else 'low' " +
                     ' end as "classification" ' +
            ' from ticket_flights tf ' +
            ' join flights f on f.flight_id = tf.flight_id ' +
            ' group by flight_no) as foo ' +
            ' group by classification ' +
            ' order by 2')
data = pd.read_sql(query, con=conn_string)
data

,classification,count
0,high,25
1,middle,76
2,low,382


# Exercise 9

In [33]:
# --9.Выведите пары городов между которыми расстояние более 5000 км
# --d = arccos {sin(latitude_a)·sin(latitude_b) + cos(latitude_a)·cos(latitude_b)·cos(longitude_a - longitude_b)}, 
# --где latitude_a и latitude_b — широты, 
# --longitude_a, longitude_b — долготы данных пунктов, 
# --d — расстояние между пунктами измеряется в радианах длиной дуги большого круга земного шара.
# --Расстояние между пунктами, измеряемое в километрах, определяется по формуле:
# --L = d·R, где R = 6371 км — средний радиус земного шара.
# --9. Output pairs of cities between which the distance is more than 5000 km
# --d = arccos {sin(latitude_a) sin(latitude_b) + cos(latitude_a) cos(latitude_b) cos(longitude_a - longitude_b)},
# --where latitude_a and latitude_b are latitudes,
# --longitude_a, longitude_b are longitudes of these points,
# --d — the distance between the points is measured in radians, the length of the arc of a great circle of the globe.
# --The distance between the points, measured in kilometers, is determined by the formula:
# --L = d R, where R = 6371 km is the average radius of the globe.

schema_name = 'bookings'
cursor.execute(f'SET search_path TO {schema_name};')
query = (   ' select city1, city2, dist ' +
            ' from (select ad1.city as "city1", ad1.longitude as "long", ad1.latitude as "lat", ' +
            ' 		ad2.city as "city2", ad2.longitude as "long", ad2.latitude as "lat", ' +
            ' 	    round((acos(sind(ad1.latitude)*sind(ad2.latitude) + ' +
            ' 	    cosd(ad1.latitude)*cosd(ad2.latitude)*cosd(ad1.longitude - ad2.longitude)))*6371) as "dist" ' +
            ' 	    from airports ad1, airports ad2 ' +
            ' 	    where ad1.city != ad2.city) as foo ' +
            ' where dist > 5000 ' +
            ' order by 3')
data = pd.read_sql(query, con=conn_string)
data

,city1,city2,dist
0,Барнаул,Анадырь,5005.0
1,Анадырь,Барнаул,5005.0
2,Улан-Удэ,Геленджик,5010.0
3,Геленджик,Улан-Удэ,5010.0
4,Комсомольск-на-Амуре,Пермь,5010.0
...,...,...,...
1017,Анапа,Петропавловск-Камчатский,7750.0
1018,Петропавловск-Камчатский,Геленджик,7761.0
1019,Геленджик,Петропавловск-Камчатский,7761.0
1020,Петропавловск-Камчатский,Сочи,7778.0
